In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
import contractions
import demoji
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.naive_bayes import MultinomialNB

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tarang\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tarang\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('Original Data.csv')
df

,rewire_id,text,label_sexist,label_category,label_vector
0,sexism2022_english-7358,"Damn, this writing was pretty chaotic",not sexist,none,none
1,sexism2022_english-2367,"Yeah, and apparently a bunch of misogynistic v...",not sexist,none,none
2,sexism2022_english-3073,How the FUCK is this woman still an MP!!!???,not sexist,none,none
3,sexism2022_english-14895,Understand. Know you're right. At same time I ...,not sexist,none,none
4,sexism2022_english-4118,Surprized they didn't stop and rape some women,not sexist,none,none
...,...,...,...,...,...
13995,sexism2022_english-14185,complexes like the 'nice chicks' that go after...,sexist,2. derogation,2.1 descriptive attacks
13996,sexism2022_english-18629,"""GRAPHIC Germany - Muslim ""refugee"" stabbing h...",not sexist,none,none
13997,sexism2022_english-3320,Lol I imagine there would be simps that are li...,not sexist,none,none
13998,sexism2022_english-13309,"It's not, the girls I go on dates with don't k...",not sexist,none,none


In [3]:
df['text'] = df['text'].str.replace('`', "'") #consistent quotes
df['text'] = df['text'].str.replace('“', '"') #consistent quotes
df['text'] = df['text'].str.replace('”', '"') #consistent quotes

for i in range(len(df.axes[0])):
    df.iat[i,1] = contractions.fix(df.iat[i,1]) #remove contractions

In [4]:
df['text'] = df['text'].str.replace('[0-9]*', "", regex=True) #remove digits
df['text'] = df['text'].str.replace('\\[USER\\]|\\[URL\\]', "", regex=True) #remove "URL" and "USER"
df['text'] = df['text'].str.lower() #convert to lowercase

In [5]:
df['text'] = df['text'].str.replace(r'\u200d', "", regex=True)
df['text'] = df['text'].str.replace(r'\u200f', "", regex=True)
df['text'] = df['text'].str.replace(r'\u200b', "", regex=True)

In [6]:
punctuation = string.punctuation + "—"
stop_words = stopwords.words('english')
porter = PorterStemmer()

for i in range(len(df.axes[0])):
    text = df.iat[i,1]
    text = text.translate(str.maketrans(punctuation, ' '*len(punctuation), '')) #remove punctuations
    text = demoji.replace(text, "")
    text = word_tokenize(text) #tokenize
    text = [word for word in text if word not in stop_words] #remove stopwords
    text = [porter.stem(word) for word in text] #stemming
    df.iat[i,1] = text
    
df

,rewire_id,text,label_sexist,label_category,label_vector
0,sexism2022_english-7358,"[damn, write, pretti, chaotic]",not sexist,none,none
1,sexism2022_english-2367,"[yeah, appar, bunch, misogynist, virgin, one, ...",not sexist,none,none
2,sexism2022_english-3073,"[fuck, woman, still, mp]",not sexist,none,none
3,sexism2022_english-14895,"[understand, know, right, time, know, enough, ...",not sexist,none,none
4,sexism2022_english-4118,"[surpriz, stop, rape, women]",not sexist,none,none
...,...,...,...,...,...
13995,sexism2022_english-14185,"[complex, like, nice, chick, go, bad, boy, nah...",sexist,2. derogation,2.1 descriptive attacks
13996,sexism2022_english-18629,"[graphic, germani, muslim, refuge, stab, young...",not sexist,none,none
13997,sexism2022_english-3320,"[lol, imagin, would, simp, like, deserv, ugli,...",not sexist,none,none
13998,sexism2022_english-13309,"[girl, go, date, kiss, first, date, text, back...",not sexist,none,none


In [7]:
class NaiveBayes():
    def __init__(self):
        self.prior = None
        self.col_probs = None
        

    def fit(self, X, y):
        feature_probs = {}
        prior_probs = {}

        for row, label in zip(X, y):
            if label not in feature_probs:
                prior_probs[label] = 0
                feature_probs[label] = [1] * len(X[0])

            prior_probs[label] += (1 / len(y))

            for feature_idx, feature in enumerate(row):
                feature_probs[label][feature_idx] += feature

        self.prior = prior_probs
        
        for counts in feature_probs.values():
            for feature_idx, feature in enumerate(counts):
                counts[feature_idx] = feature / sum(counts)
        
        self.feature_probs = feature_probs
        return self
    
    
    def predict_probs(self, X):
        probs = []
        for row in X:
            row_prob = self.prior.copy()
            
            for label, feature_probs in self.feature_probs.items():
                for feature_idx, feature in enumerate(row):
                    if feature:
                        row_prob[label] *= feature_probs[feature_idx]

            probs.append(row_prob)

        return probs
    
    def predict(self, X):
        probs = self.predict_probs(X)

        preds = []
        for row_probs in probs:
            max_prob = 0
            max_prob_label = None
            
            for label, prob in row_probs.items():
                if prob > max_prob:
                    max_prob = prob
                    max_prob_label = label
                    
            preds.append(max_prob_label)

        return preds

In [8]:
def wordFrequencies(docs):
    all_words = []

    for doc in docs:
        for word in doc:
            all_words.append(word)

    all_words = nltk.FreqDist(all_words)
    return all_words

In [9]:
def kMostFrequentWords(k, all_words):
    words = list(all_words.keys())[:k]
    vocab = set()
    for word in words:
        vocab.add(word)

    vocab = {word:idx for idx, word in enumerate(vocab)}
    return vocab

In [10]:
def getWordCounts(docs, vocab):
    count_matrix = []

    for doc in docs:
        row = [0] * len(vocab)

        for word in doc:
            if word in vocab:
                row[vocab[word]] += 1

        count_matrix.append(row)

    return count_matrix

In [11]:
def getWordOccurence(docs, vocab):
    count_matrix = []

    for doc in docs:
        row = [0] * len(vocab)

        for word in doc:
            if word in vocab:
                row[vocab[word]] = 1

        count_matrix.append(row)

    return count_matrix

In [12]:
def testNaiveBayes(vocab_size, train_docs, train_labels, test_docs, test_labels):
    vocab = kMostFrequentWords(vocab_size, wordFrequencies(train_docs))
    
    X_train = getWordOccurence(train_docs, vocab)
    y_train = list(train_labels)

    X_test = getWordOccurence(test_docs, vocab)
    y_test = list(test_labels)
    
    clf = NaiveBayes()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    return metrics.accuracy_score(y_test, y_pred), metrics.f1_score(y_test, y_pred, average="macro")

In [13]:
# interval = int(len(df)*0.2)

# for size in [500, 1000, 1500, 2000, 2500, 3000]:
#     print("\nSize: ", size)
    
#     acc_sum = 0
#     f1_sum = 0
    
#     for i in range(5):
#         train_df = pd.concat([df[0:i*interval], df[(i+1)*interval:len(df)]], axis=0)
#         test_df = df[i*interval:(i+1)*interval]
    
#         acc, f1 = testNaiveBayes(size, train_df['text'], train_df['label_sexist'], test_df['text'], test_df['label_sexist'])
#         print("Accuracy: ", acc, ", F1 Score: ", f1)
#         acc_sum += acc
#         f1_sum += f1
        
#     print("Average Accuracy: ", acc_sum/5, ", Average F1 Score: ", f1_sum/5)

#### Multinomial Naive Bayes (True/False Features Set)

With feature vector of size  500, Accuracy: 80.79, F1 (Macro): 65.83<br>
With feature vector of size 1000, Accuracy: 81.70, F1 (Macro): 70.80<br>
With feature vector of size 1500, Accuracy: 81.15, F1 (Macro): 71.83<br>
With feature vector of size 2000, Accuracy: 81.06, F1 (Macro): 72.28<br>
With feature vector of size 2500, Accuracy: 80.68, F1 (Macro): 72.14<br>
With feature vector of size 3000, Accuracy: 80.80, F1 (Macro): 72.33<br>

#### Multinomial Naive Bayes (Count Features Set)

With feature vector of size  500, Accuracy: 80.86, F1 (Macro): 64.42<br>
With feature vector of size 1000, Accuracy: 81.30, F1 (Macro): 67.09<br>
With feature vector of size 1500, Accuracy: 81.60, F1 (Macro): 71.69<br>
With feature vector of size 2000, Accuracy: 81.70, F1 (Macro): 72.38<br>
With feature vector of size 2500, Accuracy: 81.42, F1 (Macro): 72.32<br>
With feature vector of size 3000, Accuracy: 82.26, F1 (Macro): 72.38<br>